<a href="https://colab.research.google.com/github/MonitSharma/Learn-Quantum-Machine-Learning/blob/main/Gaussian_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian Transformation

Here we'll learn about the basic principles of continuous variable (CV) photonic devices.


Read [this](https://strawberryfields.readthedocs.io/en/latest/) documentation to learn more on it.

## The quantum circuit
----

For this basic tutorial, we will consider a special subset of CV operations: the Gaussian transformations. We work with the following simple Gaussian circuit:


![alt text](https://pennylane.ai/qml/_images/gaussian_transformation.svg)


What is this circuit doing?

1. **We begin with one wire in the vaccum state** : Note that we use the same notation $|0⟩$ for the initial states. In a photonic CV system, this state is the *vaccum state* i.e a state with no photons in the wire.


2. **We displace the wire(qumode)** : The displacement gate linearly shifts the state of the qumode in phase space. The vacuum state is centered at the origin in phase space, while the displaced state will be centered at the point $α$.

3. **We rotate the qumode** : This is another linear transformation in phase space, albeit a rotation (by angle $ϕ$) instead of a displacement.

4. **Finally, we measure the mean photon number** : $\langle\hat{n}\rangle =
\langle\hat{a}^\dagger \hat{a}\rangle$ This quantity, which tells us the average amount of photons in the final state, is proportional to the energy of the photonic system.







----
Our aim here is to optimize the circuit parameters $(\alpha, \phi)$ such that the mean photon number is equal to one. The rotating gate is actually a *passive transformation*, meaning that it does not change the energy of the system. The displacement gate is an *active transformation*, which modifies the energy of the photonic system.

## Constructing the QNode

Import Pennylane and NumPy

In [1]:
import pennylane as qml
from pennylane import numpy as np

Next, we instantiate a device which will be used to evaluate the circuit. Because our circuit contains only Gaussian operations, we can make use of the built-in `default.gaussian` device.

In [2]:
dev_gaussian = qml.device("default.gaussian", wires=1)

After initializing the device, we can construct our quantum node. As before, we use the `qnode()` to convert our quantum function (encoded by the circuit above) into a quantum node running on the `default.gaussian` device.

In [3]:
@qml.qnode(dev_gaussian)
def mean_photon_gaussian(mag_alpha, phase_alpha, phi):
    qml.Displacement(mag_alpha, phase_alpha, wires=0)
    qml.Rotation(phi, wires=0)
    return qml.expval(qml.NumberOperator(0))

Notice that we have broken up the complex number $α$
 into two real numbers `mag_alpha` and `phase_alpha`, which form a polar representation of $α$
. This is so that the notion of a gradient is clear and well-defined.

## Optimization

As in the [qubit rotation](https://colab.research.google.com/github/MonitSharma/Learn-Quantum-Machine-Learning/blob/main/Qubit_Rotation.ipynb) tutorial, let’s now use one of the built-in PennyLane optimizers in order to optimize the quantum circuit towards the desired output. We want the mean photon number to be exactly one, so we will use a squared-difference cost function:

In [4]:
def cost(params):
    return (mean_photon_gaussian(params[0], params[1], params[2]) - 1.0) ** 2


At the beginning of the optimization, we choose arbitrary small initial parameters:

In [5]:
init_params = np.array([0.015, 0.02, 0.005], requires_grad=True)
print(cost(init_params))

0.9995500506249999


When the gate parameters are near to zero, the gates are close to the identity transformation, which leaves the initial state largely unchanged. Since the initial state contains no photons, the mean photon number of the circuit output is approximately zero, and the cost is close to one.

-----
We avoided initial parameters which are exactly zero because that corresponds to a critical point with zero gradient.

-------

Now, let’s use the `GradientDescentOptimizer`, and update the circuit parameters over 100 optimization steps.




In [6]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.1)

# set the number of steps
steps = 20
# set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    print("Cost after step {:5d}: {:8f}".format(i + 1, cost(params)))

print("Optimized mag_alpha:{:8f}".format(params[0]))
print("Optimized phase_alpha:{:8f}".format(params[1]))
print("Optimized phi:{:8f}".format(params[2]))

Cost after step     1: 0.999118
Cost after step     2: 0.998273
Cost after step     3: 0.996618
Cost after step     4: 0.993382
Cost after step     5: 0.987074
Cost after step     6: 0.974837
Cost after step     7: 0.951332
Cost after step     8: 0.907043
Cost after step     9: 0.826649
Cost after step    10: 0.690812
Cost after step    11: 0.490303
Cost after step    12: 0.258845
Cost after step    13: 0.083224
Cost after step    14: 0.013179
Cost after step    15: 0.001001
Cost after step    16: 0.000049
Cost after step    17: 0.000002
Cost after step    18: 0.000000
Cost after step    19: 0.000000
Cost after step    20: 0.000000
Optimized mag_alpha:0.999994
Optimized phase_alpha:0.020000
Optimized phi:0.005000


The optimization converges after about 20 steps to a cost function value of zero.

We observe that the two angular parameters phase_alpha and phi do not change during the optimization. Only the magnitude of the complex displacement $|α|$ affects the mean photon number of the circuit.


Next we will learn how to utilize the extensive plugin ecosystem of Pennylane, build continuous variable quantum nodes.